## Colab Setup

In [1]:
# https://opencv.org/
!apt -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print("accelerator:", accelerator)
!pip install lxml==4.0

!pip install http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

# get terminator
!pip install fastai

# may not be necessary
!apt-get install -y libtiff5-dev

# on some of the VMs PIL is borked
# I am doing this out of abundance of caution
#!CC="cc -mavx2" pip install -U --force-reinstall pillow-simd
!pip install Pillow==4.1.1
!pip install --force-reinstall scipy



libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
accelerator: cpu


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtiff5-dev is already the newest version (4.0.8-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
  Using cached https://files.pythonhosted.org/packages/a8/0b/f163da98d3a01b3e0ef1cab8dd2123c34aee2bafbb1c5bffa354cc8a1730/scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/68/1e/116ad560de97694e2d0c1843a7a0075cc9f49e922454d32f49a80eb6f1f2/numpy-1.14.5-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: numpy 1.14.5
    Uninstalling numpy-1.14.5:
      Successfully uninstalled numpy-1.14.5
  Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0


## Dogs v Cats

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *

In [9]:
# get me some data
!wget http://files.fast.ai/data/dogscats.zip
!mkdir -p data
!mv dogscats.zip data/dogscats.zip
!unzip -q -o data/dogscats.zip -d data/

--2018-07-19 17:58:24--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857214334 (818M) [application/zip]
Saving to: ‘dogscats.zip’

dogscats.zip        100%[===================>] 817.50M   112MB/s    in 7.4s    

2018-07-19 17:58:32 (110 MB/s) - ‘dogscats.zip’ saved [857214334/857214334]



In [0]:
PATH = "data/dogscats/"
sz = 224
arch = resnet34
bs = 64

In [11]:
m = arch(True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /content/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:00<00:00, 106236525.77it/s]


In [12]:
m

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNo

In [0]:
m = nn.Sequential(*children(m)[:-2], 
                  nn.Conv2d(512, 2, 3, padding=1), 
                  nn.AdaptiveAvgPool2d(1), Flatten(), 
                  nn.LogSoftmax())

In [0]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

In [0]:
learn = ConvLearner.from_model_data(m, data)

In [0]:
learn.freeze_to(-4)

In [17]:
m[-1].trainable

True

In [18]:
m[-4].trainable

True

In [19]:
learn.fit(0.01, 1)

epoch      trn_loss   val_loss   accuracy   
    0      0.15933    0.119661   0.9855    



[array([0.11966]), 0.9855]

In [0]:
learn.fit(0.01, 1, cycle_len=1)

## CAM

In [0]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [0]:
x,y = next(iter(data.val_dl))

In [0]:
x,y = x[None,1], y[None,1]
vx = Variable(x.cuda(), requires_grad=True)

In [0]:
dx = data.val_ds.denorm(x)[0]
plt.imshow(dx);

In [0]:
sfs = [SaveFeatures(o) for o in [m[-7], m[-6], m[-5], m[-4]]]

In [0]:
%time py = m(Variable(x.cuda()))

In [0]:
for o in sfs: o.remove()

In [0]:
[o.features.size() for o in sfs]

In [0]:
py = np.exp(to_np(py)[0]); py

In [0]:
feat = np.maximum(0,to_np(sfs[3].features[0]))
feat.shape

In [0]:
f2=np.dot(np.rollaxis(feat,0,3), py)
f2-=f2.min()
f2/=f2.max()
f2

In [0]:
plt.imshow(dx)
plt.imshow(scipy.misc.imresize(f2, dx.shape), alpha=0.5, cmap='hot');

## Model

In [0]:
learn.unfreeze()
learn.bn_freeze(True)

In [0]:
lr=np.array([1e-6,1e-4,1e-2])

In [0]:
learn.fit(lr, 2, cycle_len=1)

In [0]:
accuracy(*learn.TTA())

In [0]:
learn.fit(lr, 2, cycle_len=1)

In [0]:
accuracy(*learn.TTA())